I just want to get a sense of how long the likelihood evaluation takes. 

In [1]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
from time import time
from multiprocessing import cpu_count, Pool
import warnings
from itertools import izip
from os import path
from ast import literal_eval

import numpy as np
import emcee as mc
import dynesty as dyn
from functools import partial
from scipy.linalg import inv
import h5py

from pearce.emulator import OriginalRecipe, ExtraCrispy, SpicyBuffalo, NashvilleHot, LemonPepperWet

 /u/ki/swmclau2/.conda/envs/hodemulator/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning:numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.


In [3]:
def lnprior(theta, param_names, *args):
    for p, t in izip(param_names, theta):
        low, high = _emus[0].get_param_bounds(p)

        if np.isnan(t) or t < low or t > high:
            return -np.inf
    return 0




In [40]:
from time import time
def lnlike(theta, param_names, fixed_params, r_bin_centers, y, combined_inv_cov):
    t0 = time()
    param_dict = dict(izip(param_names, theta))
    param_dict.update(fixed_params)

    emu_preds = []
    t1 = time()
    for _emu, rbc in zip(_emus, r_bin_centers):
        y_bar = _emu.emulate_wrt_r(param_dict, rbc)[0]

        emu_preds.append(10**y_bar)
        #delta = y_bar - y
        #chi2 -= np.dot(delta, np.dot(combined_inv_cov, delta))
    t2 = time()
    emu_pred = np.hstack(emu_preds)

    delta = emu_pred - y
    #print delta
    out =  - np.dot(delta, np.dot(combined_inv_cov, delta))
    t3 = time()
    print t1-t0, t2-t1, t3-t2
    return out 

In [5]:
def lnprob(theta, *args):
    """
    The total liklihood for an MCMC. Mostly a generic wrapper for the below functions.
    :param theta:
        Parameters for the proposal
    :param args:
        Arguments to pass into the liklihood
    :return:
        Log Liklihood of theta, a float.
    """
    lp = lnprior(theta, *args)
    if not np.isfinite(lp):
        return -np.inf

    return lp + lnlike(theta, *args)


In [6]:
fname = '/u/ki/swmclau2/des/PearceMCMC/HOD_MDPL2_wp_ds_rmin_0.5_HOD.hdf5'

In [7]:
f = h5py.File(fname, 'r')

In [8]:
tf = f.attrs['training_file']
fixed_params = literal_eval(f.attrs['fixed_params'])
if 'rmin' in fixed_params:
    del fixed_params['rmin']
emu_hps = literal_eval(f.attrs['emu_hps'])

In [9]:
tf

array(['/u/ki/swmclau2/des/Aemulators/wp_zheng07_h/PearceWpCosmo.hdf5',
       '/u/ki/swmclau2/des/Aemulators/ds_zheng07_h/PearceDsCosmo.hdf5'],
      dtype='|S61')

In [10]:
fixed_params = fixed_params[0] if type(fixed_params) is list else fixed_params

In [11]:
global _emus
_emus = []
for t in tf:
    print t
    _emus.append(LemonPepperWet(t, fixed_params = fixed_params))#, hyperparams = emu_hps) )

/u/ki/swmclau2/des/Aemulators/wp_zheng07_h/PearceWpCosmo.hdf5
/u/ki/swmclau2/des/Aemulators/ds_zheng07_h/PearceDsCosmo.hdf5


In [12]:
chain_pnames = f.attrs['param_names']

In [13]:
n_walkers = f.attrs['nwalkers']

In [30]:
theta = f['chain'][-1, :]

In [31]:
y = f['data'][()].flatten()

In [32]:
cov = f['cov'][()]
inv_cov = inv(cov)

In [41]:
%%timeit
lnprob(theta, chain_pnames, {}, 
       [_emus[i].scale_bin_centers for i in xrange(2)], y, inv_cov)



1.97887420654e-05 0.0476112365723 0.000159978866577
0.000106811523438 0.0579030513763 0.000158071517944
5.07831573486e-05 0.0624830722809 0.000111103057861
0.00019383430481 0.0584101676941 0.000108957290649
6.41345977783e-05 0.0440258979797 0.000113010406494
5.91278076172e-05 0.0377328395844 0.000125169754028
5.10215759277e-05 0.0377469062805 0.000108957290649
6.00814819336e-05 0.0375418663025 0.000128030776978
4.91142272949e-05 0.0391318798065 0.000108957290649
5.98430633545e-05 0.0374441146851 0.000108957290649
4.6968460083e-05 0.0170390605927 5.60283660889e-05
2.78949737549e-05 0.0164730548859 5.60283660889e-05
2.28881835938e-05 0.0165579319 5.60283660889e-05
2.31266021729e-05 0.016569852829 6.103515625e-05
2.78949737549e-05 0.0166471004486 5.60283660889e-05
2.28881835938e-05 0.0164740085602 5.50746917725e-05
2.19345092773e-05 0.0165810585022 5.50746917725e-05
2.78949737549e-05 0.0164911746979 6.29425048828e-05
2.40802764893e-05 0.0165188312531 5.50746917725e-05
2.31266021729e-05 0.